In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mgtwr.sel_bw import Sel_BW
from mgtwr.model import Model
import pandas as pd
import multiprocessing as mp
import psutil

d:\program\anaconda\envs\SMGTWR\lib\site-packages\libpysal\cg\alpha_shapes.py:39: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def nb_dist(x, y):
d:\program\anaconda\envs\SMGTWR\lib\site-packages\libpysal\cg\alpha_shapes.py:165: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_faces(triangle):
d:\program\anaconda\envs\SMGTWR\lib\site-packages\libpysal\cg\a

In [3]:
del Sel_BW, Model
from mgtwr.sel_bw import Sel_BW
from mgtwr.model import Model

In [6]:
data = pd.read_csv('data\\STNWR运行数据集.csv')
coords = data[['Lon', 'Lat', 'T']].values
X = data[['X1','X2','X3','X4','X5','X6','X7','X8']].values
X = np.log(X + 1e-20)
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
y0 = data[['Y3']].values
y = np.log(y0 + 1e-20)
print(np.mean(y, axis=0),np.std(y, axis=0))
y = (y - np.mean(y, axis=0)) / np.std(y, axis=0)

[10.61557363] [1.76259362]


GTWR

In [9]:
sel = Sel_BW(coords, y, X, mode='gtwr', spherical=True, fixed=False)
print('searching...')
with mp.Pool(psutil.cpu_count()) as pool:
    bw, tau = sel.search(bw_min=150,tau_min=0,tau_max=1e6, verbose=True, max_iter=500)
print('bw:', bw, 'tau:', tau)

searching...
Bandwidth:      521.0, tau: 381970.00000, score: 1763.233
Bandwidth:      521.0, tau: 618030.00000, score: 1763.233
Bandwidth:      749.0, tau: 381970.00000, score: 2041.130
Bandwidth:      749.0, tau: 618030.00000, score: 2041.130
Bandwidth:      379.0, tau: 236068.91910, score: 1524.509
Bandwidth:      379.0, tau: 381970.00000, score: 1524.509
Bandwidth:      521.0, tau: 236068.91910, score: 1763.233
Bandwidth:      292.0, tau: 145901.08090, score: 1308.455
Bandwidth:      292.0, tau: 236068.91910, score: 1308.455
Bandwidth:      379.0, tau: 145901.08090, score: 1524.509
Bandwidth:      237.0, tau: 90171.24503, score: 1130.038
Bandwidth:      237.0, tau: 145901.08090, score: 1130.038
Bandwidth:      292.0, tau: 90171.24503, score: 1308.455
Bandwidth:      204.0, tau: 55729.83587, score: 993.422
Bandwidth:      204.0, tau: 90171.24503, score: 993.422
Bandwidth:      237.0, tau: 55729.83587, score: 1130.038
Bandwidth:      183.0, tau: 34442.71046, score: 891.096
Bandwidth:

In [10]:
gtwr = Model(coords, y, X, bw=bw, tau=tau, mode='gtwr', spherical=True, fixed=False)
with mp.Pool(int(psutil.cpu_count() / 2)) as pool:
    gtwr_results = gtwr.fit(pool=pool)

In [11]:
print(gtwr_results.summary)

                               Model type:  Gaussian
                   Number of observations:  1120
                     Number of covariates:  9

                           Global Regression Results                            
--------------------------------------------------------------------------------
                  Residual sum of squares:  571.406
                           Log-likelihood:  -1212.340
                                      AIC:  2442.680
                                     AICc:  2444.879
                                      BIC:  -7229.018
                                       CV:  0.510
                                       R2:  0.490
                              Adjusted R2:  0.486
    Variable          Est.             SE          t(Est/SE)        p-value     
---------------- --------------- --------------- --------------- ---------------
       X0            0.000           0.021           0.000           1.000      
       X1            0.414    

In [12]:
predy = gtwr_results.predy
pd.DataFrame(predy).to_csv('C:\\Users\\YihengSu\\Desktop\\预测.csv', index=False, header=False)
pd.DataFrame(y).to_csv('C:\\Users\\YihengSu\\Desktop\\真实.csv', index=False, header=False)

In [ ]:
predy = gtwr_results.predy.reshape(10, 112)
predy = np.exp(predy * 0.32837856 -0.45641049)
residual = predy - y0.reshape(10, 112)
pd.DataFrame(residual).to_csv('C:\\Users\\YihengSu\\Desktop\\强度矩阵\\y2残差.csv', index=False, header=False)

In [20]:
pm = gtwr_results.params
pd.DataFrame(pm).to_csv('C:\\Users\\YihengSu\\Desktop\\强度矩阵\\y2系数.csv', index=False, header=False)